In [2]:
# !pip install docling_core

In [3]:
import torch
from docling_core.types.doc import DoclingDocument
from docling_core.types.doc.document import DocTagsDocument
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
import os
from tqdm.auto import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

processor = AutoProcessor.from_pretrained("ds4sd/SmolDocling-256M-preview")
model = AutoModelForVision2Seq.from_pretrained(
    "ds4sd/SmolDocling-256M-preview",
    torch_dtype=torch.bfloat16,
).to(DEVICE)

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": "Convert this page to docling."}
        ]
    },
]

FOLDER = "../data/images/-AIST-2D---Results-of-flight-tests-and-applica_2023_The-Egyptian-Journal-of-"
images = [Image.open(os.path.join(FOLDER, path)) for path in tqdm(os.listdir(FOLDER))]

for path in tqdm(os.listdir(FOLDER)):
    image = load_image(os.path.join(FOLDER, path))
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)

    generated_ids = model.generate(**inputs, max_new_tokens=8192)
    prompt_length = inputs.input_ids.shape[1]
    trimmed_generated_ids = generated_ids[:, prompt_length:]
    doctags = processor.batch_decode(
        trimmed_generated_ids,
        skip_special_tokens=False,
    )[0].lstrip()

    doctags_doc = DocTagsDocument.from_doctags_and_image_pairs([doctags], [image])
    doc = DoclingDocument(name="Document")
    doc.load_from_doctags(doctags_doc)

with open(f"{FOLDER}.md", 'w', encoding = 'utf-8') as f:
    f.write(doc.export_to_markdown())

ImportError: cannot import name 'DocTagsDocument' from 'docling_core.types.doc.document' (e:\pi_school\eva-work-notes\text-extraction\venv\Lib\site-packages\docling_core\types\doc\document.py)